# Preprocessing the data

In [1]:
# Readthe csv data and crate the dataframe
from pathlib import Path
import pandas as pd

path = Path("Resources/combined_data.csv")
df = pd.read_csv(path)
df.dropna(inplace=True)

In [2]:
# slice features and target
x = df.drop(columns=['Fraud Reporting Count', 'Quarter'])
y = df["Fraud Reporting Count"]
display(x.head())
display(y.head())

,Year,Net Operating Income(Billions),Lagged Fraud Reporting Count,Lagged Net Operating Income(Billions)
1,2014,39.8,82364.0,36.8
2,2014,38.1,95516.0,39.8
3,2014,36.0,94084.0,38.1
4,2015,39.0,90868.0,36.0
5,2015,42.6,90566.0,39.0


1    95516
2    94084
3    90868
4    90566
5    96365
Name: Fraud Reporting Count, dtype: int64

In [3]:
#Slicing data into train and test subsets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1)
x_train.shape

(23, 4)

In [4]:
#Scaling data
from sklearn.preprocessing  import StandardScaler

scaler = StandardScaler()
x_scaler = scaler.fit(x_train)
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

# Decision Tree Model

#### Training

In [5]:
#Train the decisin tree model
from sklearn import tree
model = tree.DecisionTreeClassifier()
model = model.fit(x_train_scaled, y_train)

In [6]:
#Perform the predicitons
y_pred = model.predict(x_test_scaled)

#### Results

In [12]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

display(confusion_matrix(y_test, y_pred))
display(accuracy_score(y_test, y_pred))



array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

0.0